# IMDbのDataLoaderを実装
本当は日本語の感情分析用データセットを使いたいが，最適なデータセットがないため，英語の映画レビューデータセットであるIMDb(Internet Movie Database)データセットを使用する

## IMDbデータの用意
torchtextにIMDbは用意されているが，今後も手持ちのデータで自然言語処理の実装が行えるよう，生のテキストデータを使うことにする  

aclImddディレクトリ下には，test,trainディレクトリがあり，それぞれneg,posディレクトリ下にネガティブな評価の文(4/10以下)，ポジティブな評価の文(7/10以上)がtxt形式で入っている

データの用意についての構造を以下にまとめておく  
- テキストファイル(コーパス)を用意
- tsv形式でテキストと特徴量(ラベル)をまとめて保存
- 前処理関数tokenizer_with_preprocessingを定義
    - preprocessing_text: 改行消したり，区切り文字消したりする
    - tokenizer_punctuation: スペースで単語を区切り，token化(IDを割り振る)する
- torchtext.data.Field(列)処理用関数を定義
    - テキストにはtokenize=tokenizer_with_processingを指定
- torchtext.data.TabularDataset.splitsで，作成したtsvファイルを処理
    - Field処理用関数を通し，Text, Labelのkeyと対応するvalueを持つデータのリストを取得
    - train_val_ds
    - test_ds
- train_val_dsをさらに8:2に分割
    - train_ds
    - val_ds
- torchtext.vocab.Vectorsで英語版fasttextによる単語のベクトル表現モデルを取得
- 上記で作成したtorchtext.data.Fieldのbuild_vocabメソッドでボキャブラリ(itos, stoi)を構築
    - train_dsに含まれていて10回以上使われている単語のベクトル表現のみをenglish_fasttext_vectorsから取得
    - TEXT.vocab.vectors[id]で単語のfasttextベクトル表現を取得できる
- torchtext.data.Iteratorでhoge_dsからDataLoaderを作成
    - DataLoaderはデータセットからIterableなバッチの集合を作成するので，あとはepochループ内で回すだけ

## IMDbデータセットをtsv形式に変換
IMDbデータを文章とラベル(0,1)からなるtsv形式に変換する  
文章中のタブは消去する

In [1]:
import glob, os, io
data_dir = "../../datasets/ptca_datasets/chapter7"
imdb_dir = os.path.join(data_dir, "aclImdb")
cache_dir = os.path.join(data_dir, "vector_cache")

In [2]:
def make_text_label_tsv(input_dir, output_path, label, output_mode='a'):
    with open(output_path, output_mode, encoding="utf-8") as output_file:
        for fname in glob.glob(os.path.join(input_dir, '*.txt')):
            with io.open(fname, 'r', encoding="utf-8") as input_file:
                text = input_file.readline()
                text = text.replace('\t', " ")
                text = f"{text}\t{label}\t\n"
                output_file.write(text)

In [7]:
output_path = os.path.join(imdb_dir, "IMDb_train.tsv")
input_dir = os.path.join(imdb_dir, "train", "pos")
make_text_label_tsv(input_dir, output_path, label=1, output_mode='w')
input_dir = os.path.join(imdb_dir, "train", "neg")
make_text_label_tsv(input_dir, output_path, label=0)

output_path = os.path.join(imdb_dir, "IMDb_test.tsv")
input_dir = os.path.join(imdb_dir, "test", "pos")
make_text_label_tsv(input_dir, output_path, label=1, output_mode='w')
input_dir = os.path.join(imdb_dir, "test", "neg")
make_text_label_tsv(input_dir, output_path, label=0)

imdb_dirにIMDb_train.tsvとIMDb_test.tsvが作成される

## 前処理と単語分割の関数を定義
- 改行コード`<br/>`の削除
- ピリオドとカンマ以外の記号をスペースに変えて除去
- 単語分割は半角スペースで行う

In [3]:
import string, re
print("区切り文字一覧：", string.punctuation)

区切り文字一覧： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [4]:
def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)
    
    # カンマ，ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p ==","):
            # ピリオドとカンマの前後にはスペースを入れる
            text = text.replace(p, f" {p} ")
        else:
            text = text.replace(p, " ")
    
    return text

def tokenizer_punctuation(text):
    # スペースで単語分割を行う
    return text.strip().split()

def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    tokens = tokenizer_punctuation(text)
    return tokens

print(tokenizer_with_preprocessing('I like cats.'))

['I', 'like', 'cats', '.']


## DataLoaderの作成
7.2節と大体同じだが，`init_token="<cls>", eos_token="<eos>"`を加える  
普通init_tokenはbos(beggining of sentence)だが，あとでclassを意味していた方がいいことになるらしい

In [5]:
import torchtext

max_length = 256
TEXT = torchtext.data.Field(
    sequential=True,
    tokenize=tokenizer_with_preprocessing,
    use_vocab=True,
    lower=True, # 小文字化
    include_lengths=True,
    batch_first=True,
    fix_length=max_length,
    init_token="<cls>",
    eos_token="<eos>"
)

LABEL = torchtext.data.Field(
    sequential=False,
    use_vocab=False
)

In [6]:
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path=imdb_dir,
    train='IMDb_train.tsv',
    test='IMDb_test.tsv',
    format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)]
)

print('訓練/検証データの数:', len(train_val_ds))
print('1つ目の訓練/検証データ:', vars(train_val_ds[0]))

訓練/検証データの数: 25000
1つ目の訓練/検証データ: {'Text': ['wow', ',', 'alot', 'of', 'reviews', 'for', 'the', 'devils', 'experiment', 'are', 'here', '.', 'wonderful', '.', 'my', 'name', 'is', 'steve', 'and', 'i', 'run', 'unearthed', 'films', '.', 'we', 'just', 'started', 'releasing', 'the', 'guinea', 'pig', 'films', 'on', 'dvd', 'for', 'north', 'america', '.', 'now', 'before', 'you', 'ask', 'why', 'am', 'i', 'writing', 'a', 'review', 'instead', 'ask', 'why', 'some', 'people', 'bash', 'it', '.', 'i', 'm', 'writing', 'this', 'review', 'because', 'i', 'love', 'the', 'guinea', 'pig', 'films', '.', 'why', 'do', 'i', 'love', 'em', ',', 'it', 's', 'because', 'they', 'go', 'for', 'the', 'throat', 'and', 'they', 'don', 't', 'let', 'go', '.', 'i', 've', 'seen', 'it', 'all', '.', 'almost', 'every', 'horror', 'film', 'known', 'to', 'man', ',', 'argento', ',', 'fulci', ',', 'bava', ',', 'buttgereit', '.', 'from', 'every', 'underground', 'cult', 'sensation', 'to', 'every', 'hollywood', 'blockbuster', '.', 'i', 've',

さらにtrain_val_dsを訓練データとvalidationデータに8:2で分ける  

In [7]:
import random

train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))
print("訓練データの数:", len(train_ds))
print("検証データの数:", len(val_ds))
print("1つ目の訓練データ:", vars(train_ds[0]))

訓練データの数: 20000
検証データの数: 5000
1つ目の訓練データ: {'Text': ['i', 'think', 'micheal', 'ironsides', 'acting', 'career', 'must', 'be', 'over', ',', 'if', 'he', 'has', 'to', 'star', 'in', 'this', 'sort', 'of', 'low', 'budge', 'crap', '.', 'surely', 'he', 'could', 'do', 'better', 'than', 'waste', 'his', 'time', 'in', 'this', 'rubbish', '.', 'this', 'movie', 'could', 'be', 'far', 'better', ',', 'if', 'it', 'had', 'a', 'good', 'budget', ',', 'but', 'it', 'shows', 'repeatedly', 'through', 'out', 'the', 'movie', '.', 'there', 'is', 'one', 'scene', 'at', 'a', 'outpost', ',', 'which', 'looks', 'like', ',', 'its', 'outside', 'the', 'front', 'of', 'a', 'railway', 'station', ',', 'and', 'i', 'bet', 'it', 'was', '.', 'there', 'is', 'one', 'scene', 'which', 'made', 'give', 'this', 'movie', 'a', '3', ',', 'and', 'it', 'shows', 'the', 'space', 'craft', 'landing', 'and', 'taking', 'off', 'over', 'a', 'lake', ',', 'surrounded', 'by', 'forests', '.', 'this', 'was', 'well', 'done', ',', 'but', 'the', 'rest', 'of', 't

## ボキャブラリーを作成
分散表現には英語版のfastTextであるwiki-news-300d-1M.vecを使用する  

In [9]:
from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(
    name=os.path.join(data_dir, "wiki-news-300d-1M.vec"),
    cache=cache_dir
)

print("1単語を表現する次元数:", english_fasttext_vectors.dim)
print("単語数:", len(english_fasttext_vectors.itos))

100%|█████████▉| 999586/999994 [01:31<00:00, 11105.80it/s]

1単語を表現する次元数: 300
単語数: 999994


In [10]:
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

print(TEXT.vocab.vectors.shape)

print(TEXT.vocab.vectors)

# print(TEXT.vocab.stoi)

torch.Size([17901, 300])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0786,  0.0097,  0.0023,  ...,  0.0901,  0.0283,  0.0346],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0147, -0.0415,  0.0006,  ...,  0.0387, -0.0181, -0.0128]])


最後にDataLoaderを作成する  
データセットではベクトル表現ではなく単語IDであり，大量のメモリ消費を防いでいる．  
学習時はモデル側でIDに応じてベクトル表現を取り出す

In [11]:
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=24, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=24, train=False, sort=False)

batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[  2, 416,  36,  ..., 353,   5,   3],
        [  2,   4, 114,  ..., 394,  13,   3],
        [  2,  14, 246,  ...,   1,   1,   1],
        ...,
        [  2,  15,  11,  ...,   1,   1,   1],
        [  2,  15,  24,  ...,   1,   1,   1],
        [  2,  41, 923,  ...,  16,   4,   3]]), tensor([256, 256,  96, 256, 149, 135, 256, 256, 145, 256, 192, 256, 256, 164,
        239, 256, 155, 156, 256, 178, 151, 102, 217, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


100%|█████████▉| 999586/999994 [01:50<00:00, 11105.80it/s]

以上により，IMDbの各DataLoaderと訓練データの単語を使用したボキャブラリーの分散ベクトルを用意できた．  
以上の内容はutils/dataloader.pyに用意し，次節で使用できるようにする  
次節では以上のDataLoaderと単語ベクトルを使用し，文章のネガポジ感情分析を実現するTransformerを実装する